In [1]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta
from datetime import datetime



## Utility Functions

### Generate Process Instances
This function manages the creation of process instances for a specific process variant. A process variant is simply the list of tasks in execution order, including any loops. For example ['Wake Up', 'Breakfast', 'Work', 'Dinner' 'Sleep']

In [ ]:
def generate_process_instances(process_variant, qty):

    global start_date_time
    global instance_counter
    task_list = []
    for x in range(0, qty):
        tasks = build_task_list(instance_counter, process_variant, start_date_time, task_duration_df)
        for task in tasks:
            task_list.append(task)

        # Increment the start time by 24 hour
        start_date_time = start_date_time + timedelta(hours=24)
        # Increment the process_id
        instance_counter = instance_counter +1
        
    return task_list

## Build Task List
The build_task_list function creates a list of tasks for a specific instance of a process. The sequence of tasks is defined in the variable called process_variant. Random task durations are calculated using the task_durations_df that contains the task name, the average duration and the max duration.  

This code generates a random duration by leveraging the numpy lognormal function that gives a random
number drawn from a log normal distribution. Look up a picture of log normal distributions and you'll see 
why this is useful for generating random durations based on a mean and std deviation.
most samples are near the average with a long tail stretching towards infinity.


In [ ]:
def build_task_list(instance_id, process_variant, start_date_time, task_duration_df):
    instance_task_list = []
    rnd = np.random.default_rng()  # a random number generator

    # process_variant is a series
    for task_type in process_variant:
        # get the avg and max durations from the task_df dataframe using task_type as the key
        avg_dur = task_duration_df.loc[task_type, 'Avg']
        max_dur = task_duration_df.loc[task_type, 'Max']

        sigma = (max_dur - avg_dur) / max_dur  # std dev
        log_mean = np.log(avg_dur)  # can't pass the mean duration into lognormal until it has been logged itself 
        delta = rnd.lognormal(log_mean, sigma)  # get a random sample from a log normal distribution with a std dev
        end_time = start_date_time + timedelta(hours=delta)
        task = [instance_id, task_type, start_date_time, end_time]
        instance_task_list.append(task)

        # set the start time of the next tasks to be 5 mins after the end time of this task
        start_date_time = end_time + timedelta(minutes=5)

    return instance_task_list

## Delay Task Start Time
This function shifts the task start time for a specific task with a specific attribute set to a specific value. For example, to delay the start time of breakfast by 1hour "task_list, User, Gerry, Eat Breakfast, 1"

This function will apply the same delay to every subsequent task of the process instance. 

In [ ]:
def delayStartTime(tasks, target_attribute, target_value, target_task, timeShift):

    # Logic requires we sort by process_id and start_date
    tasks = tasks.sort_values(['process_id', 'start_time'])
    tasks = tasks.reset_index(drop=True)
    process_being_modified = -1

    # iterate through the dataframe using the index value
    for x in tasks.index:
        row = tasks.loc[x]
        task_attribute = row[target_attribute]
        row_task = row['task']
        current_process = row['process_id']


        # Have we found the target activity
        if row_task == target_task and task_attribute == target_value:
            process_being_modified = current_process

        # Are we still processing the same process that we were when we found the target activity ?
        # If so we are moving all subsequent tasks back by the value of the timeshift parameter,
        # this means shifting the start time and the end time
        if current_process == process_being_modified:
            # shift the start time
            current_start = row['start_time']
            shifted_start = current_start + timedelta(hours=timeShift)
            tasks.loc[x, 'start_time'] = shifted_start
            
            # shift the end time
            current_end = row['end_time']
            shifted_end = current_end + timedelta(hours=timeShift)
            tasks.loc[x, 'end_time'] = shifted_end
            
    return tasks

## Increase Duration
This will increase the duration of the target task for each process and alter the timings of any subsequent tasks
When iterating over tasks there are four potential conditions:
1. We haven't found the target task in the tasks of a process - do nothing.
2. We have found the target task so just increase the end time
3. We have found a task the follows the target task, increase the start time and end time by the timedelta

This logic treats repeat target tasks just like any other task that follows the first target, it only delays the first encounter with 
the target task. 

In [ ]:
def increaseDuration(tasks, target_attribute, target_value, target_task, timeShift):

    # Logic requires we sort by process_id and start_date
    tasks = tasks.sort_values(['process_id', 'start_time'])
    tasks = tasks.reset_index(drop=True)
    process_being_modified = -1
    target_task_counter = 0
    
    # iterate through the dataframe using the index value
    for x in tasks.index:
        row = tasks.loc[x]
        task_attribute = row[target_attribute]
        row_task = row['task']
        current_process = row['process_id']
        processing_target_task = False
        
        # reset the target task counter when we come across a new process
        if current_process !=  process_being_modified:
            target_task_counter = 0

        # Have we found a task of the target type with a target attribute matching the target value?
        if row_task == target_task and task_attribute == target_value:
            process_being_modified = current_process
            target_task_counter = target_task_counter + 1
            
            # If this is the first target task in this process then
            # just increase the duration by increasing the end_time
            if target_task_counter == 1: 
                current_end = row['end_time']
                shifted_end = current_end + timedelta(hours=timeShift)
                tasks.loc[x, 'end_time'] = shifted_end
            else:
                current_start = row['start_time']
                shifted_start = current_start + timedelta(hours=timeShift)
                tasks.loc[x, 'start_time'] = shifted_start
            
                current_end = row['end_time']
                shifted_end = current_end + timedelta(hours=timeShift)
                tasks.loc[x, 'end_time'] = shifted_end
                
        # If we are processing ANY subsequent task that follows the target task 
        # this means shifting the start time and the end time.
        elif target_task_counter > 0:
                current_start = row['start_time']
                shifted_start = current_start + timedelta(hours=timeShift)
                tasks.loc[x, 'start_time'] = shifted_start
            
                current_end = row['end_time']
                shifted_end = current_end + timedelta(hours=timeShift)
                tasks.loc[x, 'end_time'] = shifted_end
            
    return tasks

# Initialise Data Generator

## Load Task Duration Config
For each task type that could be generated we need an average duration and a maximum duration.

In [ ]:
data = [['New Client Onboarding Request', 0.5,1.5], 
        ['Review Documents', 0.25,  2.5], 
        ['Automated Scoreboarding', 0.1, 0.15],
        ['Manual Scoreboarding', 1.0, 3.0],
        ['Update Backend Systems', 0.25, 0.5],
        ['Notification Review Request Completed', 0.1, 0.15]
       ]

task_duration_df = pd.DataFrame(data, columns=['Task', 'Avg', 'Max'])
task_duration_df.set_index('Task', inplace=True)

In [ ]:
task_duration_df.head(10)

## Reset Counters

In [ ]:
start_date_time = datetime(2017, 11, 28, 18, 00, 00)
instance_time_offset = 24
instance_counter = 1


## Create Happy Path Instances

In [ ]:

qty = 900
happy_path = ["New Client Onboarding Request", 
               "Review Documents", 
               "Automated Scoreboarding",
               "Update Backend Systems", 
               "Notification Review Request Completed"]

happy_path_task_list = []
happy_path_task_list = generate_process_instances(happy_path, qty)
happy_path_task_list_df = pd.DataFrame(happy_path_task_list)

# Increment the start time by 24 hour
start_date_time = start_date_time + timedelta(hours=24)

In [ ]:
happy_path_task_list_df.head()

## Create Instances Requiring Manual Scoreboarding

In [ ]:

qty = 80
manual_path = ["New Client Onboarding Request", 
               "Review Documents", 
               "Automated Scoreboarding",
               "Manual Scoreboarding",
               "Update Backend Systems", 
               "Notification Review Request Completed"]

manual_path_task_list = []
manual_path_task_list = generate_process_instances(manual_path, qty)
manual_path_task_list_df = pd.DataFrame(manual_path_task_list)

# Increment the start time by 24 hour
start_date_time = start_date_time + timedelta(hours=24)

In [ ]:
manual_path_task_list_df.head()

## Create Instance that loop
Create processes that loop back to Review Docs from Manual Scoreboarding.

In [ ]:
qty = 20
loop_path = ["New Client Onboarding Request", 
               "Review Documents", 
               "Automated Scoreboarding",
               "Manual Scoreboarding",
               "Review Documents",
               "Update Backend Systems", 
               "Notification Review Request Completed"]

loop_path_task_list = []
loop_path_task_list = generate_process_instances(loop_path, qty)
loop_path_task_list_df = pd.DataFrame(loop_path_task_list)

# Increment the start time by 24 hour
start_date_time = start_date_time + timedelta(hours=24)

In [ ]:
# Can only concat two dataframes at a time
df = pd.concat([happy_path_task_list_df,manual_path_task_list_df], axis=0)
df = pd.concat([df,loop_path_task_list_df], axis=0)

In [ ]:
df.columns = ['process_id', 'task', 'start_time', 'end_time']

In [ ]:
df.info()

In [ ]:
# Confirm how many processes
len(df["process_id"].unique())

In [ ]:
df.head(10)

## Add Task Level Business Data : User

In [ ]:

# Add a column for User
df["user"] = ""

In [ ]:
def setRandomUser(row):
    match row["task"]:
        case "Review Documents":
            return random.choice(['Rod','Jane','Freddy'])
        case "New Client Onboarding Request":
            return random.choice(['Clive','Francis','Nick','Seb','Tom'])
        case "Manual Scoreboarding":
            return random.choice(['Sharon','Susan', 'Sam'])
        case "Update Backend Systems":
            return "RPA"
        case "Automated Scoreboarding":
            return "SYSTEM"
        case "Notification Review Request Completed":
            return "SYSTEM"
        case _:
            return row["user"]

In [ ]:
df["user"] = df.apply(setRandomUser, axis=1)

In [ ]:
df.head(20)

## Add Task Level Business Data : UserGroup

In [ ]:
# Add a column for UserGroup
df["user_group"] = ""

In [ ]:
def setUserGroup(row):
    match row["task"]:
        case "Review Documents":
            return "Operations"
        case "New Client Onboarding Request":
            return "Sales" 
        case "Manual Scoreboarding":
            return "Risk"
        case "Update Backend Systems":
            return "SYSTEM"
        case "Automated Scoreboarding":
            return "SYSTEM"
        case "Notification Review Request Completed":
            return "SYSTEM"
        case _:
            return ""

In [ ]:
df["user_group"] = df.apply(setUserGroup, axis=1)

In [ ]:
df.head(20)

## Add Process Instance Business Data : Industry
Industy won't change during the process so all tasks for a given process ID must have the same value

In [ ]:
df["industry"] = ""

In [ ]:
process_id_list = df["process_id"].unique()

for process_id in process_id_list:
    ind = random.choice(['Federal','Finance','Healthcare','Insurance','Telecom'])

    # Update all rows in the df where the process_id = process_id setting industry to the value of ind
    df.loc[df["process_id"].eq(process_id), "industry"] = ind
    
                          
    

In [ ]:
df.head(20)

## Add Process Instance Business Data : Service Charge
Service charge will be a random choice based on industry.

In [ ]:
df["service_charge"] = 0

In [ ]:
def getServiceChargeByIndustry(industry):
    match industry:
        case "Federal":
            return random.choice([3000, 6000, 8000])
        case "Finance":
            return random.choice([10000, 12000, 20000])
        case "Healthcare":
            return random.choice([15000, 20000, 25000])
        case "Insurance":
            return 45000
        case "Telecom":
            return 49000
        case _:
            return 64000

In [ ]:
last_process = -1
service_charge = 0

process_id_list = df["process_id"].unique()

for process_id in process_id_list:
        # get the tasks for this process instance, use copy() to make it clear to the interpreter we know
        # this is a copy to supress any warnings about updating copies
        process_instance_tasks = df.loc[df.process_id == process_id].copy()
        # Get the fir row out of the task list and get the industry value
        industry = process_instance_tasks.iloc[0]['industry']
        
        # call function to get a semi-random service charge for the industry
        service_charge = getServiceChargeByIndustry(industry)
        
        # set the service charge in every row for this process instance
        df.loc[df.process_id == process_id, 'service_charge'] = service_charge
    


In [ ]:
df.head(50)

# Stories
Make a copy of the dataframe so we can restart here if required.

In [ ]:
story_df = df.copy()

## Story : 1 - Rod is Too Busy
Rod is always very busy doing other work, so he may not be able to start right away.
Lets take a look at two `Review Documents` tasks that have been done by Rod. 

In [ ]:
filter = (story_df['task'] == 'Review Documents') & (story_df['user'] == 'Rod')
story_df[filter].head(2)

Delay the start time of Rod's `Review Documents` tasks by 1 hour

In [ ]:
# If user is Rod and activity is `Review Documents` shift the start_time by 1 hour
# 
story_df = delayStartTime(story_df, 'user','Rod', 'Review Documents', 1)

Lets see the result of this change:

In [ ]:
filter = (story_df['task'] == 'Review Documents') & (story_df['user'] == 'Rod')
story_df[filter].head(2)

## Story : 2 - Complex Doc Review
Increase the time taken for doc review by 2hrs when the industry is federal. 
The rationale here is that the documentation requirements are greater.

Lets look at some `Review Documents` where the industry is `Federal`

In [ ]:
filter = (story_df['task'] == 'Review Documents') & (story_df['industry'] == 'Federal')
story_df[filter].head(2)

In [ ]:
story_df[story_df['process_id'] == 4]

In [ ]:
story_df = increaseDuration(story_df, 'industry','Federal', 'Review Documents', 2)

In [ ]:
story_df[story_df['process_id'] == 4]

## Export Finished Dataset

In [ ]:
story_df.to_csv('pm_tasks_1k.csv', index=False, date_format='%d-%m-%Y %H:%M:%S')

In [ ]:
Just some cells for inspecting the data